# Grafazo: Con que otros alumnos puedo hacer TPs en un futuro?

Este grafo se encarga de analizar "camadas" de gente: alumnos que cursaron juntos y les fue parecido en la facu. La idea final es que yo me haga compañero de alumnos que tiendan a cursar las mismas materias que yo, y que tengamos el mismo nivel académico.

Ojo, este grafo no apunta a responder *en que* materia hacer tps juntos: eso involucraría fijarse que curso cada alumno y fijarse que personas todavía no cursaron lo mismo. La idea es un poco mas generalizada a encontrar compañeros de clase, no importa en que materia. Ya al ser de la misma camada sabemos que nos quedan materias en las que nos vamos a cruzar

### Cómo es el grafo?

- Nodos: alumnos
- Aristas: conectar dos alumnos que hayan cursado la misma materia el mismo cuatrimestre
- Peso de las aristas: correlacion entre notas

Por ejemplo: como juampi y yo cursamos las mismas materias por dos años y siempre nos sacamos notas parecidas, probablemente somos de la misma camada de alumnos

In [ ]:
import pandas as pd

df = pd.read_pickle('fiuba-map-data.pickle')
df.sample(3)

In [ ]:
df_nodes_metadata = df[df.columns & ["Padron", "Carrera", "aplazos", "optativas", "checkboxes"]]
df_nodes_metadata = df_nodes_metadata[df_nodes_metadata["aplazos"].notnull() | df_nodes_metadata["optativas"].notnull()]
df_nodes_metadata = df_nodes_metadata.groupby(["Padron", "Carrera"], as_index=False).first()

df_nodes = df[["Padron", "Carrera", "Orientacion", "Final de Carrera"]]
df_nodes = df_nodes.drop_duplicates()
df_nodes = df_nodes.merge(df_nodes_metadata, how="outer")
df_nodes.set_index("Padron", inplace=True)
display(df_nodes.dropna().tail(3))

from itertools import combinations
df_edges = (df[df['materia_cuatrimestre'].notnull()]
     .groupby(['materia_id', 'materia_cuatrimestre'])[['Padron', 'materia_nota']]
     .apply(lambda x : list(combinations(x.values,2)))
     .apply(pd.Series)
     .stack()
     .reset_index(level=0, name='Usuarios')
)

df_edges = df_edges.reset_index()
df_edges[['src', 'dst']] = df_edges['Usuarios'].tolist()
df_edges[['src_padron', 'src_nota']] = df_edges['src'].tolist()
df_edges[['dst_padron', 'dst_nota']] = df_edges['dst'].tolist()
# Nos quedamos solo con las materias aprobadas (nota > 0)
df_edges = df_edges[df_edges['src_nota'] > 0]
df_edges = df_edges[df_edges['dst_nota'] > 0]
df_edges = df_edges[['src_padron', 'dst_padron', 'materia_cuatrimestre', 'materia_id', 'src_nota', 'dst_nota']]
display(df_edges.dropna().tail(3))

In [ ]:
import networkx as nx
G = nx.from_pandas_edgelist(df_edges, 
                            source='src_padron', 
                            target='dst_padron', 
                            edge_attr=['materia_id','materia_cuatrimestre', 'src_nota', 'dst_nota'], 
                            create_using=nx.MultiGraph())

nx.set_node_attributes(G, df_nodes.to_dict('index'))
print(G)

In [ ]:
# Veamos el grafo
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(30,10))
ax = axes.flatten()

nx.draw_networkx(G, pos=nx.circular_layout(G), width=0.005, node_size=50, with_labels=False, ax=ax[0])
nx.draw_networkx(G, pos=nx.spiral_layout(G),   width=0.005, node_size=50, with_labels=False, ax=ax[1])

In [ ]:
# # Solo como demo: ahora veamos el grafo, pero dibujemos todas las aristas
# # Robadisimo de: https://stackoverflow.com/a/60638452
# pos = nx.random_layout(G)
# nx.draw_networkx_nodes(G, pos)
# ax = plt.gca()
# for e in G.edges:
#     ax.annotate("",
#                 xy=pos[e[0]], xycoords='data',
#                 xytext=pos[e[1]], textcoords='data',
#                 arrowprops=dict(arrowstyle="-", color="0.5",
#                                 connectionstyle="arc3,rad=rr".replace('rr',str(0.3*e[2]))),
#                 )
# plt.axis('off')
# plt.show()

## Stats generales del grafo

In [ ]:
print(f"""
  El diámetro de la red: {nx.diameter(G)}
  El grado promedio de la red: {sum([n[1] for n in G.degree()]) / len(G):.2f}
  TODO: Los allegados promedio de la red: 
  Puentes globales: {list(nx.bridges(G))}
""")

## Comunidades

In [ ]:
from networkx.algorithms import community
louvain = community.louvain_communities(G)

In [ ]:
from networkx.algorithms import community
import matplotlib.colors as mcolors
import random

random.seed(42)
plt.figure(figsize=(30,10))

louvain = community.louvain_communities(G)
draw_nodes = {}
colors = random.sample(list(mcolors.TABLEAU_COLORS), len(louvain))
for louvaincommunity, color in zip(louvain, colors):
    draw_nodes.update({n: color for n in louvaincommunity})
    
plt.title("Louvain Communities")
nx.draw_networkx(G, 
                 nodelist=draw_nodes.keys(), 
                 node_color=list(draw_nodes.values()), 
                 width=0.005, 
                 pos=nx.kamada_kawai_layout(G),
                 font_size=10)

### Evaluación de comunidades por "generación"
¿Existe correlación entre la distribución de padrones y la comunidad? De tal forma que las comunidades sean por camadas o "generacionales"

In [ ]:
louvain_padrones = []
for i, comunidad in enumerate(louvain):
    for padron in comunidad:
        louvain_padrones.append((padron, i))
df_comunidades = pd.DataFrame(louvain_padrones, columns=["padron", "comunidad"])
df_comunidades["comunidad"].value_counts()

In [ ]:
import seaborn as sns

# len patch for overflow
df = df_comunidades[(df_comunidades['padron'].str.isdigit()) & (df_comunidades["padron"].str.len() >= 5) & (df_comunidades["padron"].str.len() <= 6)].copy()
df["padron"] = df["padron"].astype(int)

# remove outliers by percentiles, stolen from https://stackoverflow.com/a/59366409
Q1 = df["padron"].quantile(0.10)
Q3 = df["padron"].quantile(0.90)
IQR = Q3 - Q1
df_comunidades = df[~((df["padron"] < (Q1 - 1.5 * IQR)) |(df["padron"] > (Q3 + 1.5 * IQR)))]
df_comunidades.sample(3)

Se puede observar una mínima correlación considerando el intervalo más frecuente de cada comunidad

In [ ]:
# TODO: display as rows instead of columns, perhaps?
# TODO: play with the "stat" parameter. another interesting one is the "density" parameter
# TODO: Antes de graficar, ordenar las comunidades segun el promedio: asi te quedan de izquierda a derecha la comunidad mas vieja (90osos) a la comunidad mas joven (100osos)
g = sns.displot(
    df_comunidades,
    x="padron",
    col="comunidad",
    element="step",
    stat="count",
    common_norm=False,
)

### Materias y cuatrimestres en común
Se puede aproximar a una mejor noción de _camada_ que simplemente tomando el padrón, tomando el conjunto de materias/cuatrimestre más populares por comunidad.

Las comunidades considerando dicha métrica, se observan personas que:

- Entraron a la facultad en el 2019c1
- Cursaron Taller de Programación en el 2020c1
- Entraron a la facultad en el 2020c1
- Tienen planificado hacer Base de Datos y Legislación en el 2023c1 (comunidad más chica)
- Entraron a la facultad en el 2021c1

In [ ]:
df = df_comunidades.copy()
df['padron'] = df['padron'].astype(str)
df_src = pd.merge(df_edges, df, left_on='src_padron', right_on='padron')
df_dst = pd.merge(df_edges, df, left_on='dst_padron', right_on='padron')
(
    pd
        .concat([df_src, df_dst])
        .groupby(['comunidad', 'materia_cuatrimestre', 'materia_id'])
        .agg(count=('src_padron', 'count'))
        .sort_values('count', ascending=False)
        .reset_index()
        .groupby('comunidad')   # obtener top 3 materias/cuatrimestres por comunidad
        .head(3)
        .sort_values(['comunidad', 'count'])
)

## Subcomunidades

In [ ]:
subnetwork = nx.subgraph(G, max(louvain, key=len))
subnetwork_louvain = community.louvain_communities(subnetwork)
plt.figure(figsize=(20,10))

draw_nodes = {}
colors = random.sample(list(mcolors.TABLEAU_COLORS), len(subnetwork_louvain))
for louvain, color in zip(subnetwork_louvain, colors):
    draw_nodes.update({n: color for n in louvain})
    
plt.title("Louvain Communities (of the OG Louvain Community with the most nodes)")
nx.draw_networkx(subnetwork, 
                 nodelist=draw_nodes.keys(), 
                 node_color=list(draw_nodes.values()), 
                 width=0.05, 
                 pos=nx.kamada_kawai_layout(subnetwork),
                 font_size=10)

In [ ]:
subnetwork_louvain

In [ ]:
# todo: usar graphrole y rolx

In [ ]:
# todo: ejercicio 5 de parcialito 3

In [ ]:
# todo: leer TODA la documentacion de "algorithms" de nx y ver que aplica aca
# https://networkx.org/documentation/stable/reference/algorithms/index.html

In [ ]:
# - Los 3 notebooks de grafos hacen exactamente el mismo analisis, pero con distintas conclusiones
# - Cada notebook de grafo devuelve un __algo__ que sea util y valioso para el usuario (despues vemos como se lo proveemos)
#   y la respuesta puede ser o de comunidad o de roles, hay que ver que es lo adecuado.